In [ ]:
import pandas as pd
import numpy as np
import math
from gurobipy import *

In [ ]:
responses=pd.read_csv("responses.csv")

In [ ]:
responses

,Timestamp,Code name: (Cause we don't want to be biased),Area of interest:,How important is getting matched to your 'Area of interest'?,How do you rate yourself in programming skills?,How important your teammate/mates be good in coding?,How do you rate yourself in Mechanism design concepts?,How important your teammate/mates be good in mechanism design?,Preferred team size?,How important is it to work in your preferred team size?
0,2/22/2022 15:38:59,popcorn,Sports Analytics,3,4,4,2,4,2,4
1,2/22/2022 15:46:20,Sarf,Wordle Solver,3,4,2,2,3,1,4
2,2/22/2022 15:46:34,Kyle Johnson,Gerrymandering,1,3,3,3,3,3,3
3,2/22/2022 15:51:43,lalala,Sports Analytics,4,4,4,4,4,1,5
4,2/22/2022 17:48:57,Ya Ling Su,Sports Analytics,3,3,5,3,5,3,4
5,2/22/2022 18:42:21,Praveen,Social Good,4,2,5,4,3,3,4
6,2/22/2022 20:12:56,mari,Sports Analytics,5,3,4,2,4,3,4
7,2/23/2022 16:06:52,Weiyu Peng,Sports Analytics,4,4,4,3,4,2,4


In [ ]:
data=responses

In [ ]:
data.shape

(8, 10)

In [ ]:
data.dtypes

Timestamp                                                         object
Code name: (Cause we don't want to be biased)                     object
Area of interest:                                                 object
How important is getting matched to your 'Area of interest'?       int64
How do you rate yourself in programming skills?                    int64
How important your teammate/mates be good in coding?               int64
How do you rate yourself in Mechanism design concepts?             int64
How important your teammate/mates be good in mechanism design?     int64
Preferred team size?                                               int64
How important is it to work in your preferred team size?           int64
dtype: object

In [ ]:
col_names=[1,2,3,4,5,6,7,8,9,10]
data.set_axis(col_names, axis=1,inplace=True)

In [ ]:
n=data.shape[0]
n

8

In [ ]:
comp=np.zeros(shape=(n,n)) #For compatibility values
w1=np.zeros(shape=n) # For compatibilty weightage of single teams
w2=np.zeros(shape=(n,n)) #For compatibilty weightage team of 2s
w3=np.zeros(shape=(n,n,n)) #For compatibitly weightage between team of 3s

In [ ]:
weightage = {'1' : 1, '2': 5, '3':10 , '4':50, '5':250} #Weightage to the importance part of questions i.e 4,6,8,10

In [ ]:
prob={4: 1,3:0.75,2:0.5,1:0.25,0:0.01} #Rewarding the teams with huge skill set difference

In [ ]:
#Replace the rating of importance questions with our weightage matrix
for j in [4,6,8,10]:
    for i in range(n): 
        if data.loc[i,j]==1:
            data.loc[i,j] = weightage['1']
        elif data.loc[i,j] == 2:
            data.loc[i,j] = weightage['2']
        elif data.loc[i,j] == 3:
            data.loc[i,j] = weightage['3']
        elif data.loc[i,j] == 4:
            data.loc[i,j] = weightage['4']
        elif data.loc[i,j] == 5:
            data.loc[i,j] = weightage['5'] 

In [ ]:
data

,1,2,3,4,5,6,7,8,9,10
0,2/22/2022 15:38:59,popcorn,Sports Analytics,10,4,50,2,50,2,50
1,2/22/2022 15:46:20,Sarf,Wordle Solver,10,4,5,2,10,1,50
2,2/22/2022 15:46:34,Kyle Johnson,Gerrymandering,1,3,10,3,10,3,10
3,2/22/2022 15:51:43,lalala,Sports Analytics,50,4,50,4,50,1,250
4,2/22/2022 17:48:57,Ya Ling Su,Sports Analytics,10,3,250,3,250,3,50
5,2/22/2022 18:42:21,Praveen,Social Good,50,2,250,4,10,3,50
6,2/22/2022 20:12:56,mari,Sports Analytics,250,3,50,2,50,3,50
7,2/23/2022 16:06:52,Weiyu Peng,Sports Analytics,50,4,50,3,50,2,50


In [ ]:
#Compatibilty calculations

for i in range(n):
    for j in range(n):
        num=0
        den=0
        for k in [3,9]:
            if data.loc[i,k]==data.loc[j,k]: #For focus area and team size num summation and denominator summation are same if same preferences
                num=data.loc[i,k+1]+num
            den=data.loc[i,k+1]+den
        for k in [5,7]: #For skill set difference 
            diff=abs(data.loc[i,k]-data.loc[j,k])
            num=prob[diff]*data.loc[i,k+1]+num
            den=data.loc[i,k+1]+den
        comp[i,j]=num/den #Compatibility of i over j
                 
            
comp       

array([[0.38125   , 0.00625   , 0.15625   , 0.221875  , 0.21875   ,
        0.3125    , 0.14375   , 0.45625   ],
       [0.002     , 0.802     , 0.05      , 0.734     , 0.05      ,
        0.1       , 0.018     , 0.034     ],
       [0.16129032, 0.16129032, 0.36129032, 0.16129032, 0.32903226,
        0.48387097, 0.40645161, 0.08387097],
       [0.18875   , 0.68875   , 0.0625    , 0.7525    , 0.1875    ,
        0.06375   , 0.21875   , 0.1575    ],
       [0.24107143, 0.22321429, 0.09821429, 0.24107143, 0.11607143,
        0.3125    , 0.22321429, 0.13392857],
       [0.36111111, 0.36111111, 0.31944444, 0.3475    , 0.31944444,
        0.285     , 0.32638889, 0.35416667],
       [0.6575    , 0.0325    , 0.1575    , 0.71875   , 0.7825    ,
        0.21875   , 0.7525    , 0.6875    ],
       [0.565     , 0.065     , 0.065     , 0.315     , 0.315     ,
        0.1875    , 0.375     , 0.505     ]])

In [ ]:
#Weightage value for 2 persons scenarios
for i in range(n):
    for j in range(n):
        w2[i,j]=math.sqrt(comp[i,j]*comp[j,i])
w2

array([[0.38125   , 0.00353553, 0.15875016, 0.20464336, 0.22963966,
        0.33592741, 0.30743394, 0.50772163],
       [0.00353553, 0.802     , 0.08980265, 0.71101512, 0.10564428,
        0.19002924, 0.02418677, 0.04701064],
       [0.15875016, 0.08980265, 0.36129032, 0.10040242, 0.17976559,
        0.39315378, 0.25301409, 0.07383504],
       [0.20464336, 0.71101512, 0.10040242, 0.7525    , 0.21260502,
        0.14883926, 0.39651805, 0.22273864],
       [0.22963966, 0.10564428, 0.17976559, 0.21260502, 0.11607143,
        0.31595314, 0.41792963, 0.20539596],
       [0.33592741, 0.19002924, 0.39315378, 0.14883926, 0.31595314,
        0.285     , 0.26720324, 0.2576941 ],
       [0.30743394, 0.02418677, 0.25301409, 0.39651805, 0.41792963,
        0.26720324, 0.7525    , 0.5077524 ],
       [0.50772163, 0.04701064, 0.07383504, 0.22273864, 0.20539596,
        0.2576941 , 0.5077524 , 0.505     ]])

In [ ]:
for i in range(n):
    for j in range(n):
        if(i==j):
            w1[i]=w2[i,j]

In [ ]:
w1

array([0.38125   , 0.802     , 0.36129032, 0.7525    , 0.11607143,
       0.285     , 0.7525    , 0.505     ])

In [ ]:
#Final weight matrix
for i in range(n):
    for j in range(n):
        for k in range(n):
            w3[i,j,k]=(w2[i,j]*w2[j,k]*w2[i,k])**(1./3.)
w3

array([[[0.38125   , 0.01682829, 0.21259158, 0.25180735, 0.27191587,
         0.35040227, 0.33029689, 0.46148089],
        [0.01682829, 0.02156229, 0.03693908, 0.08012668, 0.04410109,
         0.06088453, 0.02973456, 0.04386239],
        [0.21259158, 0.03693908, 0.20881493, 0.1483037 , 0.18713537,
         0.27574481, 0.2311374 , 0.18121777],
        [0.25180735, 0.08012668, 0.1483037 , 0.31586435, 0.21538032,
         0.21709686, 0.29219367, 0.28497469],
        [0.27191587, 0.04410109, 0.18713537, 0.21538032, 0.18292499,
         0.28993791, 0.30900627, 0.28824038],
        [0.35040227, 0.06088453, 0.27574481, 0.21709686, 0.28993791,
         0.31801327, 0.3021897 , 0.35290558],
        [0.33029689, 0.02973456, 0.2311374 , 0.29219367, 0.30900627,
         0.3021897 , 0.41432077, 0.42954614],
        [0.46148089, 0.04386239, 0.18121777, 0.28497469, 0.28824038,
         0.35290558, 0.42954614, 0.50681279]],

       [[0.01682829, 0.02156229, 0.03693908, 0.08012668, 0.04410109,
         

In [ ]:
model = Model('Teams')

Restricted license - for non-production use only - expires 2023-10-25


In [ ]:
# Variables: x_{i} binary representing whether person i is in a single person team
#x = model.addVars([i for i in range(n)] ,vtype = GRB.BINARY, name = 'x')
# Variables: y_{i,j} binary representing whether persons i,j are in a two people team
#y = model.addVars([(i,j) for i in range(n) for j in range(n)], vtype = GRB.BINARY, name = 'y')
# Variables: z_{i,j,k} binary representing whether persons i,j,k are in a thre people team
#z = model.addVars([(i,j,k) for i in range(n) for j in range(n) for k in range(n)], vtype = GRB.BINARY, name = 'z')

In [ ]:
# Constraint: Each person can be matched to at only one team
#c1 = model.addConstrs(x[i]+quicksum(y[i,j] for j in range(n))+quicksum(z[i,j,k] for j in range(n) for k in range(n)) == 1 for i in range(n))

In [ ]:
# Objective: Maximize total weightage
#model.setObjective(quicksum(x[i]* w1[i] for i in range(n)) + quicksum(y[i,j]* w2[i,j] for i in range(n) for j in range(n)) + quicksum(z[i,j,k]* w3[i,j,k] for i in range(n) for j in range(n) for k in range(n)), GRB.MAXIMIZE)

In [ ]:
# Variables: x_{i,j,k} binary representing whether persons i,j,k are in a thre people team
x = model.addVars([(i,j,k) for i in range(n) for j in range(n) for k in range(n)], vtype = GRB.BINARY, name = 'x')

In [ ]:
# Constraint: Each person can be matched to at only one team
c1 = model.addConstrs(quicksum(x[i,j,k] for j in range(n) for k in range(n)) == 1 for i in range(n))
c2 = model.addConstrs(quicksum(x[i,j,k] for i in range(n) for k in range(n)) == 1 for j in range(n))
c3 = model.addConstrs(quicksum(x[i,j,k] for i in range(n) for j in range(n)) == 1 for k in range(n))

In [ ]:
# Constraint equations needs to be rechecked.

In [ ]:
# Objective: Maximize total weightage
model.setObjective(quicksum(x[i,j,k]* w3[i,j,k] for i in range(n) for j in range(n) for k in range(n)), GRB.MAXIMIZE)

In [ ]:
model.params.outputflag = 0
model.optimize()
model.Params.LogToConsole = 0


In [ ]:
teams=[]
for v in model.getVars():
  if v.X > 0:
    teams.append(v.varName[2:-1].split(','))
    #print(v.varName,v.X)

In [ ]:
teams

[['0', '7', '0'],
 ['1', '1', '1'],
 ['2', '2', '2'],
 ['3', '3', '3'],
 ['4', '5', '4'],
 ['5', '4', '5'],
 ['6', '6', '6'],
 ['7', '0', '7']]

In [ ]:
#printing a final output with names would look better!